In [1]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.style
import matplotlib as mpl

mpl.style.use('classic')

import baraffe_tables
from baraffe_tables.table_search import baraffe_table_search

from astropy.table import Table
from astropy import units as u
from astropy.constants import G

import mesa_helper as mh
import os
import shutil

%matplotlib inline

In [2]:
mJtomSun = u.jupiterMass.to(u.solMass)
mJtoGrams = u.jupiterMass.to(u.g)

rJtorSun = u.jupiterRad.to(u.solRad)
rJtoCm = u.jupiterRad.to(u.cm)

print(mJtomSun)
print(mJtoGrams)
print(rJtorSun)
print(rJtoCm)


0.0009545942339693249
1.8981245973360504e+30
0.10276268506540176
7149200000.0


In [3]:
print(5*rJtoCm)

35746000000.0


In [4]:
#jupiter masses
Minit = np.array((20.0, 14.0, 9.8, 6.9, 4.801, 3.4, 2.3, 1.6, 1.1, 0.82, 0.57, 0.4, 0.28, 0.2))

#jupiter radii
Rinit = np.array((5.0, 2.0, 1.5, 1.0, 0.75, 0.5))
#Rinit = np.array((2.0, 1.75, 1.5, 1.25, 1.0, 0.75, 0.5, 0.25, 0.1))
#Rinit = np.array((0.75, 0.5))

In [5]:
# make directories for all planet initial masses
os.chdir('/Users/emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_24.08.1')
    
for i, m in enumerate(Minit):
    os.system("rm -r Minit={0}Mj".format(m))
    os.system("cp -r template_makeplanet Minit={0}Mj".format(m))

#edit inlist_create, inlist_evolve, rn
for i, m in enumerate(Minit):
    print(m)
    #edit inlists
    create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])
    evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])
    pulse_data_filename = "{0}_Mj_{1}_Rj.mesa".format(m, Rinit[0])
    save_profile_filename = "{0}_Mj_{1}_Rj.profile".format(m, Rinit[0])

    inlist_create = mh.Inlist('./Minit={0}Mj/inlist_create'.format(m))
    inlist_evolve = mh.Inlist('./Minit={0}Mj/inlist_evolve'.format(m))

    #edit inlist_create options
    inlist_create.set_option('mass_in_gm_for_create_initial_model', m*mJtoGrams)
    # start with initial radius equal to 5 Rj. This will need to be revised downward for less massive planets.
    # If it needs to be revised to be very small, may also need to change kap_lowT_prefix option.
    inlist_create.set_option('radius_in_cm_for_create_initial_model', Rinit[0]*rJtoCm)
    inlist_create.set_option('save_model_filename', create_model_filename)
    inlist_create.set_option('initial_model_relax_num_steps', 50)

    #edit inlist_evolve options
    inlist_evolve.set_option('load_model_filename', create_model_filename)
    inlist_evolve.set_option('save_model_filename', evolve_model_filename)
    #inlist_evolve.set_option('mesh_delta_coeff', 1.0) #1.0 is default
    inlist_evolve.set_option('save_pulse_data_filename', pulse_data_filename)
    inlist_evolve.set_option('filename_for_profile_when_terminate', save_profile_filename)
    if m <= 1.1:
        inlist_evolve.set_option('irradiation_flux', 5.e4)
        inlist_evolve.set_option('column_depth_for_irradiation', 500.0)
    else:
        inlist_evolve.set_option('irradiation_flux',0)
        inlist_evolve.set_option('column_depth_for_irradiation',-1)

    # edit rn script
    f = open('./Minit={0}Mj/rn'.format(m), 'r')
    g = f.read()
    f.close()
    #print(g)
    g = g.replace("<<create_model_filename>>", create_model_filename)
    g = g.replace("<<evolve_model_filename>>", evolve_model_filename)
    
    h = open('./Minit={0}Mj/rn_temp'.format(m), 'w')
    h.write(g)
    h.close()
    shutil.copyfile('./Minit={0}Mj/rn_temp'.format(m),'./Minit={0}Mj/rn'.format(m))
    
    # execute mk script
    os.chdir('./Minit={0}Mj/'.format(m))
    os.system('./mk'.format(m))
    os.chdir('..')


20.0
Set mass_in_gm_for_create_initial_model to 3.79625d31
Set radius_in_cm_for_create_initial_model to 3.5746d10
Set save_model_filename to 'planet_create_20.0_Mj_5.0_Rj.mod'
Set initial_model_relax_num_steps to 50
Set load_model_filename to 'planet_create_20.0_Mj_5.0_Rj.mod'
Set save_model_filename to 'planet_evolve_20.0_Mj_5.0_Rj.mod'
Set save_pulse_data_filename to '20.0_Mj_5.0_Rj.mesa'
Set filename_for_profile_when_terminate to '20.0_Mj_5.0_Rj.profile'
Set irradiation_flux to 0
Set column_depth_for_irradiation to -1
gfortran -fopenmp -o ../star  run_star_extras.o run_star.o  run.o  -L/Users/emily/mesa-24.08.1/lib -lstar -lgyre_mesa -lgyre -lionization -latm -lcolors -lturb -lstar_data -lnet -leos -lkap -lrates -lneu -lchem -linterp_2d -linterp_1d -lnum -lauto_diff -lforum -lmtx -lconst -lmath -lutils `mesasdk_crmath_link` `mesasdk_lapack95_link` `mesasdk_lapack_link` `mesasdk_blas_link` `mesasdk_hdf5_link`  `mesasdk_pgplot_link` -lz 


ld: warning: ignoring duplicate libraries: '-lblas', '-llapack', '-lz'


14.0
Set mass_in_gm_for_create_initial_model to 2.65737d31
Set radius_in_cm_for_create_initial_model to 3.5746d10
Set save_model_filename to 'planet_create_14.0_Mj_5.0_Rj.mod'
Set initial_model_relax_num_steps to 50
Set load_model_filename to 'planet_create_14.0_Mj_5.0_Rj.mod'
Set save_model_filename to 'planet_evolve_14.0_Mj_5.0_Rj.mod'
Set save_pulse_data_filename to '14.0_Mj_5.0_Rj.mesa'
Set filename_for_profile_when_terminate to '14.0_Mj_5.0_Rj.profile'
Set irradiation_flux to 0
Set column_depth_for_irradiation to -1
gfortran -fopenmp -o ../star  run_star_extras.o run_star.o  run.o  -L/Users/emily/mesa-24.08.1/lib -lstar -lgyre_mesa -lgyre -lionization -latm -lcolors -lturb -lstar_data -lnet -leos -lkap -lrates -lneu -lchem -linterp_2d -linterp_1d -lnum -lauto_diff -lforum -lmtx -lconst -lmath -lutils `mesasdk_crmath_link` `mesasdk_lapack95_link` `mesasdk_lapack_link` `mesasdk_blas_link` `mesasdk_hdf5_link`  `mesasdk_pgplot_link` -lz 


ld: warning: ignoring duplicate libraries: '-lblas', '-llapack', '-lz'


0.4
Set mass_in_gm_for_create_initial_model to 7.5925d29
Set radius_in_cm_for_create_initial_model to 3.5746d10
Set save_model_filename to 'planet_create_0.4_Mj_5.0_Rj.mod'
Set initial_model_relax_num_steps to 50
Set load_model_filename to 'planet_create_0.4_Mj_5.0_Rj.mod'
Set save_model_filename to 'planet_evolve_0.4_Mj_5.0_Rj.mod'
Set save_pulse_data_filename to '0.4_Mj_5.0_Rj.mesa'
Set filename_for_profile_when_terminate to '0.4_Mj_5.0_Rj.profile'
Set irradiation_flux to 5d4
Set column_depth_for_irradiation to 5d2
gfortran -fopenmp -o ../star  run_star_extras.o run_star.o  run.o  -L/Users/emily/mesa-24.08.1/lib -lstar -lgyre_mesa -lgyre -lionization -latm -lcolors -lturb -lstar_data -lnet -leos -lkap -lrates -lneu -lchem -linterp_2d -linterp_1d -lnum -lauto_diff -lforum -lmtx -lconst -lmath -lutils `mesasdk_crmath_link` `mesasdk_lapack95_link` `mesasdk_lapack_link` `mesasdk_blas_link` `mesasdk_hdf5_link`  `mesasdk_pgplot_link` -lz 


ld: warning: ignoring duplicate libraries: '-lblas', '-llapack', '-lz'


0.28
Set mass_in_gm_for_create_initial_model to 5.31475d29
Set radius_in_cm_for_create_initial_model to 3.5746d10
Set save_model_filename to 'planet_create_0.28_Mj_5.0_Rj.mod'
Set initial_model_relax_num_steps to 50
Set load_model_filename to 'planet_create_0.28_Mj_5.0_Rj.mod'
Set save_model_filename to 'planet_evolve_0.28_Mj_5.0_Rj.mod'
Set save_pulse_data_filename to '0.28_Mj_5.0_Rj.mesa'
Set filename_for_profile_when_terminate to '0.28_Mj_5.0_Rj.profile'
Set irradiation_flux to 5d4
Set column_depth_for_irradiation to 5d2
gfortran -fopenmp -o ../star  run_star_extras.o run_star.o  run.o  -L/Users/emily/mesa-24.08.1/lib -lstar -lgyre_mesa -lgyre -lionization -latm -lcolors -lturb -lstar_data -lnet -leos -lkap -lrates -lneu -lchem -linterp_2d -linterp_1d -lnum -lauto_diff -lforum -lmtx -lconst -lmath -lutils `mesasdk_crmath_link` `mesasdk_lapack95_link` `mesasdk_lapack_link` `mesasdk_blas_link` `mesasdk_hdf5_link`  `mesasdk_pgplot_link` -lz 


ld: warning: ignoring duplicate libraries: '-lblas', '-llapack', '-lz'


In [6]:
def update_radius(m, rold, rnew):
    """
    m in Mj, rnew in Rj
    Assumes that this will be run from within the relevant model folder
    """
    
    #edit inlists
    old_create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, rold)
    old_evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, rold)

    create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, rnew)
    evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, rnew)
    pulse_data_filename = "{0}_Mj_{1}_Rj.mesa".format(m, rnew)
    save_profile_filename = "{0}_Mj_{1}_Rj.profile".format(m, rnew)


    inlist_create = mh.Inlist('./inlist_create')
    inlist_evolve = mh.Inlist('./inlist_evolve')

    #edit inlist_create options
    inlist_create.set_option('mass_in_gm_for_create_initial_model', m*mJtoGrams)
    inlist_create.set_option('radius_in_cm_for_create_initial_model', rnew*rJtoCm)
    inlist_create.set_option('save_model_filename', create_model_filename)
    

    #edit inlist_evolve options
    inlist_evolve.set_option('load_model_filename', create_model_filename)
    inlist_evolve.set_option('save_model_filename', evolve_model_filename)
    inlist_evolve.set_option('save_pulse_data_filename', pulse_data_filename)
    inlist_evolve.set_option('filename_for_profile_when_terminate', save_profile_filename)
    
    # edit rn script
    f = open('./rn', 'r')
    g = f.read()
    f.close()

    g = g.replace(old_create_model_filename, create_model_filename)
    g = g.replace(old_evolve_model_filename, evolve_model_filename)
    
    h = open('./rn_temp', 'w')
    h.write(g)
    h.close()
    shutil.copyfile('./rn_temp','./rn')
    
    return


In [7]:
pwd

'/Users/emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_24.08.1'

In [8]:
# need to try stepping through R values until code runs
for i, m in enumerate(Minit):
    os.chdir('/Users/emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_24.08.1')
    
    print("")
    print("m is {0} Mj".format(m))
    os.system('pwd')
    os.chdir('./Minit={0}Mj/'.format(m))
    os.system('pwd')

    print("r is {0} Rj".format(Rinit[0]))
    create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])
    evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])

    os.system('./rn')

    #test if "create" and "evolve" models exist, indicating success
    test_create = os.path.isfile(create_model_filename)
    test_evolve = os.path.isfile(evolve_model_filename)

    print("test_create is {0} and test_evolve is {1}".format(test_create, test_evolve))

    if test_create is True and test_evolve is True:
        #success, no need to try further r values
        continue
    else:
        print("model did not converge; stepping through decreasing radii")
        for j, r in enumerate(Rinit[:-1]):
            os.system('rm ./LOGS/*')
            os.system('rm ./photos/*')

            print("trying at new radius {0} Rj".format(Rinit[j+1]))
            update_radius(m, r, Rinit[j+1])
            create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, Rinit[j+1])
            evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, Rinit[j+1])

            
            os.system('./rn')
            test_create = os.path.isfile(create_model_filename)
            test_evolve = os.path.isfile(evolve_model_filename)
            
            if test_create is True and test_evolve is True:
                #success, no need to try further r values
                print("test_create is {0} and test_evolve is {1}".format(test_create, test_evolve))
                break 
            
    


m is 20.0 Mj
/Users/emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_24.08.1
/Users/emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_24.08.1/Minit=20.0Mj
r is 5.0 Rj
run inlist_create_header
DATE: 2024-09-11
TIME: 14:49:42
 reading user weak rate file /Users/emily/mesa-24.08.1/data/rates_data/rate_tables/S13_r_be7_wk_li7.h5
 version_number r24.08.1
 read inlist_create
 create initial model
                         create initial model iteration           1
      log10(L/Lsun)=  -2.97569028E+00
               Mass=   1.77441956E+30
             Radius=   6.66204450E+10
 
      log10(L/Lsun)=  -3.07996188E+00
               Mass=   1.63670084E+30
             Radius=   6.39715627E+10
 
      log10(L/Lsun)=  -2.25626134E+00
               Mass=   1.63841424E+30
             Radius=   6.61941755E+10
 
                         create initial model iteration           2
      log10(L/Lsun)=  -2.72925134E+00
               Mass=   3.32155444E+30
